In [1]:
import sys
sys.path.append("..")

In [2]:
from caches.query_cache import SPARQLQueryCache, ConstructionQueryCache

In [3]:
con_query_cache = ConstructionQueryCache(
    redis_host="localhost",
    redis_port=6379,
    redis_ttl=15*60,
    es_host="localhost",
    es_port=9200,
    es_index="query_cache",
    similarity_threshold=0.7
)

In [4]:
con_query_cache.search(
    "Ngày sinh của vua Quang Trung là ngày nào".lower(), "cache:query:")

2025-04-27 13:00:21.323 | INFO     | caches.query_cache:search:94 - Redis cache hit


{'sparql': 'sparql_query',
 'query_type': 'query_type',
 'template_based': False,
 'llm_generated': False,
 'entities_used': []}

In [5]:
con_query_cache.save(
    "Ngày sinh của vua Quang Trung là ngày nào", "cache:query:",
    {
        "sparql": "sparql_query",
        "query_type": "query_type",
        "template_based": False,
        "llm_generated": True,
        "entities_used": []
    }
)

In [6]:
con_query_cache.search(
    "Ngày sinh của vua Quang Trung là ngày nào".lower(), "cache:query:")

2025-04-27 13:00:21.426 | INFO     | caches.query_cache:search:94 - Redis cache hit


{'sparql': 'sparql_query',
 'query_type': 'query_type',
 'template_based': False,
 'llm_generated': False,
 'entities_used': []}

In [7]:
sparql_cache = SPARQLQueryCache(
    redis_host="localhost",
    redis_port=6379,
    redis_ttl=15*60
)

In [8]:
query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?deathDate WHERE {
    ?instance rdfs:label "Quang Trung" .
    ?instance <http://example.org/ontology/deathDate> ?deathDate .
}"""

In [9]:
sparql_cache.search(query.lower(), "cache:sparql:")

2025-04-27 13:00:21.519 | INFO     | caches.query_cache:search:39 - Redis cache hit


{'success': True,
 'execution_time': 0,
 'endpoint': 'test',
 'query_size': 340,
 'timestamp': '2025-04-27 12:55:58',
 'results': {'format': 'unknown', 'data': 'test', 'info': 'Raw results'}}

In [10]:
import time
result = {
    "success": True,
    "execution_time": 0,
    "endpoint": "test",
    "query_size": len(query),
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "results": {
        "format": "unknown",
        "data": "test",
        "info": "Raw results"
    }
}

sparql_cache.save(query, "cache:sparql:", result)

2025-04-27 13:00:21.549 | INFO     | caches.query_cache:save:59 - Cache query successfully


In [11]:
sparql_cache.search(query.lower(), "cache:sparql:")

2025-04-27 13:00:21.841 | INFO     | caches.query_cache:search:39 - Redis cache hit


{'success': True,
 'execution_time': 0,
 'endpoint': 'test',
 'query_size': 340,
 'timestamp': '2025-04-27 13:00:21',
 'results': {'format': 'unknown', 'data': 'test', 'info': 'Raw results'}}

In [12]:
from agents.query_execution import QueryExecutionAgent

2025-04-27 13:00:22,919 | INFO | nl-to-sparql | Logging initialized: console=INFO, file=DEBUG


In [13]:
agent = QueryExecutionAgent(
    redis_host="172.26.33.23",
    redis_port=6379,
    redis_ttl=15*60
)

In [14]:
agent.execute_query(
    sparql_query=query
)

2025-04-27 13:00:23.783 | INFO     | caches.query_cache:search:39 - Redis cache hit


{'success': True,
 'execution_time': 0,
 'endpoint': 'test',
 'query_size': 340,
 'timestamp': '2025-04-27 13:00:21',
 'results': {'format': 'unknown', 'data': 'test', 'info': 'Raw results'}}

In [15]:
from agents.sparql_construction import SPARQLConstructionAgent

2025-04-27 13:00:25,369 | INFO | numexpr.utils | Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-04-27 13:00:25,370 | INFO | numexpr.utils | NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'config.agent_config'